In [1]:
import os
import sys
import json
import pandas as pd
import numpy as np

if r'G:\PythonProjects\WineRecognition2' not in sys.path:
    sys.path.insert(0, r'G:\PythonProjects\WineRecognition2')
from data_master import DataLoader

In [ ]:
with open(r'G:\PythonProjects\WineRecognition2\data_info.json') as file:
    data_info = json.load(file)
all_keys = data_info['keys']['all']
all_keys.remove('Punctuation')
all_keys.remove('Other')
all_keys

In [ ]:
data_path = r'G:\PythonProjects\WineRecognition2\data\excel\new_Bruxelles_Wine_ES.xlsx'
df = DataLoader.load_excel_data(data_path)
df.head()

In [ ]:
df.drop([col for col in df.columns if not col.startswith('add')], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df.rename(columns={
    'add_originFullName': 'Add_OriginFullName',
    'add_keyWordTrue': 'Add_KeyWordTrue',
    'add_keyWordFalse': 'Add_KeywordFalse',
    'add_grapeVarieties': 'Add_GrapeVarieties',
    'add_tradeName': 'Add_TradeName',
    'add_brand': 'Add_Brand',
    'add_geoIndication': 'Add_GeoIndication',
    'add_wineType': 'Add_WineType',
    'add_bottleSize': 'Add_BottleSize',
    'add_sweetness': 'Add_Sweetness',
    'add_wineColor': 'Add_WineColor',
    'add_vintage': 'Add_Vintage'
}, inplace=True)

In [ ]:
for index, row in df.iterrows():
    if len(row['add_additionalName']) > len(row['Add_KeyWordTrue']):
        df.loc[index, 'Add_KeyWordTrue'] = row['add_additionalName']

In [ ]:
df.drop([col for col in df.columns if not col.startswith('Add')], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
translate = {
    'originFullName': 'Add_OriginFullName',
    'keyWordTrue': 'Add_KeyWordTrue',
    'keyWordFalse': 'Add_KeyWordFalse',
    'grapeVarieties': 'Add_GrapeVarieties',
    'tradeName': 'Add_TradeName',
    'brand': 'Add_Brand',
    'geoindication': 'Add_GeoIndication',
    'wineType': 'Add_WineType',
    'bottleSize': 'Add_BottleSize',
    'sweetness': 'Add_Sweetness',
    'wineColor': 'Add_WineColor',
    'vintage': 'Add_Vintage'
}

In [ ]:
new_df = []
for index, row in df.iterrows():
    splitted_fullname = row['Add_OriginFullName'].split('}')
    splitted_fullname = [token.strip().replace('{', '').split(': ') for token in splitted_fullname if token.strip()]
    keys_not_met = {key: False for key in all_keys.copy()}
    new_row = {}
    for key, value in splitted_fullname:
        if key not in translate:
            continue
        translated_key = translate[key]
        keys_not_met[translated_key] = True
        if translated_key not in new_row:
            new_row[translated_key] = value
        else:
            new_row[translated_key] = f'{new_row[translated_key]} {value}'
    for key in filter(lambda k: not keys_not_met[k], keys_not_met):
        if key not in row:
            new_row[key] = ''
            continue
        if key == 'Add_GeoIndication':
            new_row[key] = str(row[key]).split(' / ')[0]
            continue
        new_row[key] = row[key]
    new_df.append(new_row)

In [ ]:
new_df = pd.DataFrame(new_df)
new_df.head()

In [ ]:
def transform_grape(x):
    grapes = []
    for grape in x.split('; '):
        grapes.append(grape.split(', ')[0])
    return ' '.join(grapes)

new_df.Add_GrapeVarieties = new_df.Add_GrapeVarieties.apply(transform_grape)

In [ ]:
def transform_cells(x):
    return ''.join(c for c in str(x) if c not in '[]{};,')

new_df = new_df.applymap(transform_cells)

In [ ]:
new_df.to_csv(os.path.join(r'G:\PythonProjects\WineRecognition2\data\csv', 'Bruxelles_Wine_ES.csv'))

In [ ]:
new_df.to_excel(os.path.join(r'G:\PythonProjects\WineRecognition2\data\excel', 'Bruxelles_Wine_ES(transformed).xlsx'))

In [3]:
np.random.randn(1)[0]

-1.0416261940597868